# Разработка классификатора новостей

**Нужно:**
* выбрать какой-либо новостной ресурс, где к новостям привязаны категории или метки (например http://lenta.ru, http://fontanka.ru, http://gazeta.ru)
* загрузить новости по некоторому набору (5-10) категорий за пару лет
* обучить классификатор на эти новостях
* продемонстрировать его работу, разработав простеший web-интерфейс (вариант - telegram-бот), куда пользователь вводит текст новости и на выходе получает наиболее вероятную категорию. В качестве фреймворка проще всего взять [Flask](http://flask.pocoo.org) (см. примеры) .

In [20]:
import lxml.html
import lxml.etree
import requests
import pymorphy2
from tqdm import tqdm

In [21]:
def parse_category_by_url(url):
    response = requests.get(url)
    if (response.status_code == 200):
        res = []
        tree = lxml.html.fromstring(response.text)
        links = tree.xpath('//div[contains(@class, "big_center ml30 fll")]//td[contains(@class, "dom")]')
        for link in links:
            try:
                formed_link = form_url_by_suburl(link.getchildren()[0].get("href"))                
                res.append(formed_link)
            except:
                pass
        return res
        
    return []

In [22]:
def parse_news_on_page(url):
    response = requests.get(url)
    if (response.status_code == 200):
        res = []
        tree = lxml.html.fromstring(response.text)
        links = tree.xpath('//div[contains(@class, "calendar-list")]//div[contains(@class, "calendar-item-title")]')
        for link in links:
            try:
                formed_link = form_url_by_suburl(link.getchildren()[0].get("href"))
                if len(formed_link)>0:
                    res.append(formed_link)
            except:
                pass
        return res
    return []

In [23]:
def form_url_by_suburl(suburl):
    if suburl[0] == '/':
        return 'http://www.fontanka.ru' + suburl
    return ''

In [30]:
def get_content(url):
    response = requests.get(url)
    if (response.status_code == 200):        
        analyzer = pymorphy2.MorphAnalyzer()
        #words_from_query = {analyzer.normal_forms(x)[0] for x in query.split(' ')}
        tree = lxml.html.fromstring(response.text)
        text = tree.xpath('//div[contains(@class, "article_fulltext")]')
        arttitle = tree.xpath('//h1[contains(@class, "article_title")]')
        try:
            title = arttitle[0].text_content()
            #print(text[0].text_content())
            res = ''.join(text[0].text_content().replace('\t', ' ').replace('\n', ' ').splitlines())
        except:
            title = ''
            res = ['']
    return (title, res)

In [32]:
def save_cat_in_file(url, filename):
    day_links = parse_category_by_url(url)
    f = open(filename, 'w')
    for day_link in tqdm(day_links):
        news_links = parse_news_on_page(day_link)
        for news_link in news_links:
            (title, text) = get_content(news_link)
            try:
                f.write(f"{title}\t{text}\n")
            except:
                pass
    f.close()
            
save_cat_in_file('http://www.fontanka.ru/business/arc/2017/all.html', 'buis.txt')

100%|████████████████████████████████████████████████████████████████████████████████| 301/301 [05:53<00:00,  1.18s/it]
